In [ ]:
!pip install wandb

In [ ]:
import wandb
import numpy as np
import gym
from gym import spaces
from gym.wrappers import TransformReward
wandb.login()

/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.finish()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class DrunkManEnvBase(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['console']}

    def __init__(self, p=0.7, min_position=0, max_position=10, goal_position=10, penalty_case=False, H=None, discounted_reward = False, circle_mdp=False, h =0):
        super(DrunkManEnvBase, self).__init__()

        # Action space: 0 (backward), 1 (forward)
        self.action_space = spaces.Discrete(2)

        # Observation space: Position along the line
        self.observation_space = spaces.Discrete(max_position+1)

        # Environment parameters
        self.p = p
        self.min_position = min_position
        self.max_position = max_position
        self.goal_position = goal_position
        self.state = None
        self.discounted_reward = discounted_reward
        self.penalty_case = penalty_case
        self.H = H
        self.circle_mdp = circle_mdp

    def reset(self):
        self.state = self.min_position
        return self.state

    def step(self, action, h=0):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        done = bool(self.state >= self.goal_position)
        # Determine movement based on the probability
        if np.random.rand() < self.p:
            movement = 1 if action == 1 else -1
        else:
            movement = -1 if action == 1 else 1


        if self.circle_mdp and done:
          done = 0
          self.reset()
        else:
        # Update the position
          if not done:
            self.state += movement
            self.state = np.clip(self.state, self.min_position, self.max_position)
          if done:
            self.state = self.state
          done = bool(self.state >= self.goal_position)
        # Check if the goal is reached


        # Assign reward
        if self.penalty_case:
            reward = self.H if done else -1
        if self.discounted_reward:
            reward = (self.H - h)/self.H if done else 0
        elif self.circle_mdp:
            reward = (self.max_position+1)/self.H if done else 0
        else:
            reward = 1/self.H if done else 0

        info = {}

        return self.state, reward, done, info

    def render(self, mode='console'):
        if mode != 'console':
            raise NotImplementedError()
        print(f"Position: {self.state}")

    def close(self):
        pass

In [ ]:
class DrunkManEnv(object):
    def __init__(self, drift_probability=0.85, space_length=7, H=15,env_params={}):
        self.base_env = DrunkManEnvBase(p=env_params.get('drift_probability',drift_probability),
                                        max_position=env_params.get('space_length',space_length),
                                        goal_position=env_params.get('space_length',space_length),
                                        penalty_case=env_params.get('penalty_case',False),
                                        H=H,
                                        discounted_reward=env_params.get('discounted_reward',False),
                                        circle_mdp = env_params.get('circle_mdp',False))
        self.action_space = self.base_env.action_space
        self.A = self.action_space.n
        self.state_space = self.base_env.observation_space
        self.S = self.state_space.n
        self.H = H
        self.p = env_params.get('drift_probability',drift_probability)
        self.env_params = env_params
        self.pi_star = np.ones([self.H, self.S])
        self.absorbing_states = [self.S - 1]

    def compute_transition(self):
        self.P = {a:np.zeros([self.S,self.S]) for a in range(self.A)}
        self.R = {a:0 for a in range(self.A)}
        action_map = {0:'left',1:'right'}
        def action_idx_map(a, idx, max_idx):
            next_state_left = max(0, idx-1)
            next_state_right = min(idx+1, max_idx)
            if a == 0:
                next_state_list = [next_state_left, next_state_right]
            else:
                next_state_list = [next_state_right, next_state_left]
            output_dict = {next_state_list[0]:self.p, next_state_list[1]:1-self.p}
            return output_dict

        for idx in range(self.S):
            for a in range(self.A):
                P_trans = self.P[a].copy()
                if idx==self.S-1:
                  if not self.env_params.get('circle_mdp',False):
                    P_trans[idx, idx] = 1
                  else:
                    P_trans[idx, 0] = 1
                else:
                    next_state_prob_dict = action_idx_map(a,idx,self.S)
                    for s,p in next_state_prob_dict.items():
                        P_trans[idx, s] = p
                self.P[a] = P_trans

    def value_iteration(self):
        self.V_prime = np.zeros([self.H + 1, self.S])
        self.Q_prime = np.zeros([self.H, self.S, self.A])
        for h in range(self.H - 1, -1, -1):
            for s in range(self.S):
                for a in range(self.A):
                    if self.env_params.get('penalty_case',False):
                        self.Q_prime[h][s][a] = np.sum(self.P[a][s] * (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (self.H * int(s != self.S - 1)) - 1
                    elif self.env_params.get('discounted_reward',False):
                        self.Q_prime[h][s][a] = self.env_params.get('gamma',0.99) * np.sum(self.P[a][s] * (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * ((self.H - h)/self.H) * int(s != self.S - 1)
                    elif self.env_params.get('circle_mdp',False):
                        self.Q_prime[h][s][a] = (self.P[a][s] @ (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (self.S/self.H)
                    else:
                        self.Q_prime[h][s][a] = (self.P[a][s] @ (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (1/self.H)

            self.V_prime[h] = self.Q_prime[h].max(axis=1)

    def policy_evaluation(self,policy):
        V_policy = np.zeros([self.H + 1, self.S])
        Q_policy = np.zeros([self.H, self.S, self.A])
        for h in range(self.H - 1, -1, -1):
            for s in range(self.S):
                for a in range(self.A):
                    if self.env_params.get('penalty_case',False):
                        self.Q_prime[h][s][a] = np.sum(self.P[a][s] * (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (self.H * int(s != self.S - 1)) - 1
                    elif self.env_params.get('discounted_reward',False):
                        self.Q_prime[h][s][a] = self.env_params.get('gamma',0.99) * np.sum(self.P[a][s] * (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (1 * int(s != self.S - 1))
                    else:
                        self.Q_prime[h][s][a] = np.sum(self.P[a][s] * (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (1 * int(s != self.S - 1))
                V_policy[h][s] = Q_policy[h][s][policy[h,s]]

        return V_policy




In [ ]:
class FrozenLakeEnv(object):
    def __init__(self, map_name="4x4", is_slippery=True, simple_example=True, H=25,env_params={}):
        is_slippery = env_params.get('is_slippery', is_slippery)
        if env_params.get('simple_example',simple_example):
            self.base_env = gym.make('FrozenLake-v1', map_name=map_name, desc=["SFFF", "FFFF", "FFFF", "FFFG"], is_slippery=is_slippery)
        else:
            self.base_env = gym.make('FrozenLake-v1', desc = env_params.get('desc_map',["SFFF", "FHFH", "FFFH", "HFFG"]), is_slippery=is_slippery)
        self.action_space = self.base_env.action_space
        self.A = self.action_space.n
        self.state_space = self.base_env.observation_space
        self.S = self.state_space.n
        self.is_slippery = is_slippery
        self.H = H
        self.env_params = env_params
        max_row_idx = len(self.base_env.desc)
        max_col_idx = len(self.base_env.desc[0])
        self.absorbing_states = [row_idx*max_col_idx + col_idx for row_idx in range(len(self.base_env.desc)) for col_idx in range(len(self.base_env.desc[row_idx])) if self.base_env.desc[row_idx][col_idx] in [b'H', b'G']]
        if env_params.get('circle_mdp',False):
            self.base_env = TransformReward(self.base_env, lambda x: 6*x/self.H)
        elif not env_params.get('discounted_reward',False) and self.env_params.get('gamma') == 1:
            self.base_env = TransformReward(self.base_env, lambda x: x/self.H)

    def compute_transition(self):
        self.P = {a:np.zeros([self.S,self.S]) for a in range(self.A)}
        self.R = {a:0 for a in range(self.A)}
        max_row_idx = len(self.base_env.desc)
        max_col_idx = len(self.base_env.desc[0])
        action_map = {0:'left',1:'down',2:'right',3:'up'}
        def action_idx_map(a, row_idx, col_idx, is_slippery=self.is_slippery):
            next_state_left = row_idx * max_col_idx + max(0, col_idx - 1)
            next_state_down = min(max_row_idx-1, row_idx + 1) * max_col_idx + col_idx
            next_state_right = row_idx * max_col_idx + min(max_col_idx-1, col_idx + 1)
            next_state_up = max(0, row_idx - 1) * max_col_idx + col_idx
            next_state_list = [next_state_left, next_state_down,next_state_right,next_state_up]
            perpendicular_action_dict = {0:[1,3],1:[0,2],2:[1,3],3:[0,2]}
            if not is_slippery:
                output_dict = {next_state_list[a]:1}
            else:
                output_dict = {}
                for next_state in [next_state_list[a], next_state_list[perpendicular_action_dict[a][0]], next_state_list[perpendicular_action_dict[a][1]]]:
                    if next_state not in output_dict:
                        output_dict[next_state] = 1/3
                    else:
                        output_dict[next_state] = output_dict[next_state] + 1/3
            return output_dict

        for row_idx in range(len(self.base_env.desc)):
            for col_idx in range(len(self.base_env.desc[row_idx])):
                for a in range(self.A):
                    P_trans = self.P[a].copy()
                    if self.base_env.desc[row_idx][col_idx] in [b'H', b'G'] and not self.env_params.get('circle_mdp', False):
                        P_trans[row_idx*max_col_idx + col_idx, row_idx*max_col_idx + col_idx] = 1
                    elif self.base_env.desc[row_idx][col_idx] in [b'H', b'G'] and self.env_params.get('circle_mdp', False):
                        P_trans[row_idx*max_col_idx + col_idx, 0] = 1
                    else:
                        next_state_prob_dict = action_idx_map(a, row_idx, col_idx)
                        for s,p in next_state_prob_dict.items():
                            P_trans[row_idx*max_col_idx + col_idx, s] = p
                    self.P[a] = P_trans

    def value_iteration(self):
        self.V_prime = np.zeros([self.H + 1, self.S])
        self.Q_prime = np.zeros([self.H, self.S, self.A])

        for h in range(self.H - 1, -1, -1):
            for s in range(self.S):
                for a in range(self.A):
                    if self.env_params.get('penalty_case',False):
                        self.Q_prime[h][s][a] = np.sum(self.P[a][s] * (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (self.H * int(s != self.S - 1)) - 1
                    elif self.env_params.get('discounted_reward',False):
                        self.Q_prime[h][s][a] = self.env_params.get('gamma',0.99) * np.sum(self.P[a][s] * (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * ((self.H - h)/self.H) * int(s != self.S - 1)
                    elif self.env_params.get('circle_mdp',False):
                        self.Q_prime[h][s][a] = (self.P[a][s] @ (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (6/self.H)
                    else:
                        self.Q_prime[h][s][a] = (self.P[a][s] @ (self.V_prime[h + 1])) + self.P[a][s][
                            self.S - 1] * (1/self.H)

            self.V_prime[h] = self.Q_prime[h].max(axis=1)



    def policy_evaluation(self,policy):
        V_policy = np.zeros([self.H + 1, self.S])
        Q_policy = np.zeros([self.H, self.S, self.A])
        for h in range(self.H - 1, -1, -1):
            for s in range(self.S):
                for a in range(self.A):
                    if self.env_params.get('discounted_reward',False):
                        Q_policy[h][s][a] = self.env_params.get('gamma',0.99) * np.sum(self.P[a][s] * (V_policy[h + 1])) + self.P[a][s][
                        self.S - 1] * int(s != self.S - 1)
                    else:
                        Q_policy[h][s][a] = np.sum(self.P[a][s] * (V_policy[h + 1])) + self.P[a][s][
                        self.S - 1] * int(s != self.S - 1)

                V_policy[h][s] = Q_policy[h][s][policy[h,s]]

        return V_policy

In [ ]:
from tqdm import tqdm
Env_mapping = {
    'FrozenLake':FrozenLakeEnv,
    'DrunkMan':DrunkManEnv
}
class ProvablyEfficientQLearning(object):
    def __init__(self, bonus_type='Hoeffding', H=25, K=20000, c=0.01, env_type = 'FrozenLake', env_params={}):
        self.Env = Env_mapping[env_type](H=H, env_params=env_params)
        self.env_type = env_type
        self.env = self.Env.base_env
        self.A = self.Env.A
        self.S = self.Env.S
        self.H = self.Env.H
        self.K = K
        self.T = self.K * self.H
        self.max_V = self.H if env_params.get('penalty_case',False) else 1
        self.Q = np.ones([self.H, self.S, self.A]) * self.max_V
        self.V = np.ones([self.H + 1, self.S]) * self.max_V
        self.V[self.H] = np.zeros(self.S) # This one is the must
        if env_params.get('discounted_reward', False):
            for s in self.Env.absorbing_states:
                self.V[:, int(s)] = np.zeros(self.H + 1)
        self.N = np.zeros([self.H, self.S, self.A])
        self.bonus_type = bonus_type
        self.c = c # need more tuning
        self.p = .05 # need more tuning
        self.tau = np.log(self.S * self.A  *self.T / self.p) #np.log(self.S * self.A  *self.K / self.p) #
        self.logging_Q = {}
        self.gamma = 1 if not env_params.get('discounted_reward',False) else env_params.get('gamma',0.99)
        self.env_params = env_params
        self.no_early_stopping = (env_params.get('circle_mdp', False) or (not env_params.get('discounted_reward', False)) )
        if self.bonus_type == 'Bernstein':
            self.mu = np.zeros([self.H, self.S, self.A])
            self.sigma = np.zeros([self.H, self.S, self.A])
            self.betaold = np.zeros([self.H, self.S, self.A])
            self.betanew = np.zeros([self.H, self.S, self.A])
            self.c1 = self.c2 = self.c

    def learnRate(self, t):
        return (self.H + 1)/ (self.H + t)

    def Bonusterm(self, t, h,s,a,s1):
        if self.bonus_type == 'Hoeffding':
            return self.c*np.sqrt(  (self.max_V)**2 * self.tau/ t) # self.c*np.sqrt(  self.H * (self.max_V)**2 * self.tau/ t)
        elif self.bonus_type == 'Bernstein':
            self.mu[h,s,a] = self.mu[h,s,a] + self.V[h+1,s1]
            self.sigma[h,s,a] = self.sigma[h,s,a] + (self.V[h+1,s1])**2
            beta_term_111 = (self.sigma[h,s,a] - (self.mu[h,s,a])**2)/t + self.H
            beta_term_11 = np.sqrt(self.H * beta_term_111 * self.tau / t)
            beta_term_12 = np.sqrt((self.H)**7 * self.S * self.A) * self.tau / t
            beta_term_1 = self.c1 * (beta_term_11 + beta_term_12)
            beta_term_2 = self.c2 * np.sqrt( (self.H)**3 * self.tau / t)
            beta = min(beta_term_1, beta_term_2)
            self.betaold[h,s,a] = self.betanew[h,s,a]
            self.betanew[h,s,a] = beta
            alpha = self.learnRate(t)
            bonus = (beta - (1-alpha)*self.betaold[h,s,a]) / (2*alpha)
            return bonus

    def train(self):
        self.rList = []
        # for h in range(self.H):
        #     for s in range(self.S):
        #       for a in range(self.A):
        #         self.Q[h,s,a] += self.Bonusterm(1, h, s, a, s)
        self.rList_stepwise = []
        for k in tqdm(range(self.K)):
            s = self.env.reset()
            d = False
            r_k = 0
            trajectory = {}
            for h in range(self.H):
                a = np.random.choice(np.where(self.Q[h,s,:] == self.Q[h,s,:].max())[0])
                self.N[h,s,a] += 1
                t = self.N[h,s,a]
                s1, r, d, _ = self.env.step(a)
                if self.env_type == 'FrozenLake' and d and self.env_params.get('circle_mdp', False) and s == self.S-1:
                  s1 = self.env.reset()
                if self.env_type == 'FrozenLake'  and d and not self.env_params.get('discounted_reward', False) and  s == s1 == self.S-1:
                  r = self.rList_stepwise[-1]
                if d and self.env_params.get('discounted_reward', False) and s1 == self.S-1:
                  r = (self.H - h) / self.H
                trajectory[h] = (s,a,s1,r,d)
                r_k += r*(self.gamma**h)
                self.rList_stepwise.append(r*(self.gamma**h))
                if d and not self.no_early_stopping:
                  break
                s = s1


            if k%10000 == 0:
                print([trajectory[h][0] for h in range(self.H) if h in trajectory])
            for h in range(self.H - 1, -1, -1):
                if h in trajectory:
                  s,a,s1,r,d = trajectory[h]
                  t = self.N[h,s,a]
                  b = self.Bonusterm(t, h,s,a,s1)
                  # print(b)
                  lr = self.learnRate(t)
                  self.Q[h,s,a] = (1-lr) * self.Q[h,s,a] + lr*(r + self.gamma * self.V[h+1,s1]  +b)
                  self.V[h,s] = min(np.max(self.Q[h,s,:]),self.max_V)
            # self.logging_Q[k] = self.Q.copy()
            self.rList.append(r_k)
        self.trajectory = trajectory

    def stepwise_regret(self):
        self.regret_step_wise = []
        self.Env.compute_transition()
        self.Env.value_iteration()
        self.pi_star = self.Env.Q_prime.argmax(axis=2)
        total_steps = len(self.rList_stepwise)
        current_steps = 0
        current_cum_regret = 0
        while current_steps < total_steps:
            s = self.env.reset()
            d = False
            for h in range(self.H):
                a = int(self.pi_star[h][s])
                s1, r, d, _ = self.env.step(a)
                if d and self.env_params.get('discounted_reward', False) and s1 == self.S-1:
                    r = (self.H - h) / self.H
                current_cum_regret += r*(self.gamma**h) - self.rList_stepwise[current_steps]
                if current_steps % self.H == 0:
                    self.regret_step_wise.append(current_cum_regret)

                s = s1
                current_steps += 1
                if d and not self.no_early_stopping:
                    break
                if current_steps >= total_steps:
                    break
    def regret_compute(self):
        regret = []
        self.Env.compute_transition()
        self.Env.value_iteration()
        V_prime = self.Env.V_prime
        for k in tqdm(range(self.K)):
            Q_k = self.logging_Q[k]
            policy_k = Q_k.argmax(axis=2)
            eval = self.Env.policy_evaluation(policy_k)
            if regret:
                regret.append(regret[k-1]+ (V_prime[0][0] - eval[0][0]))
            else:
                regret.append( (V_prime[0][0] - eval[0][0]))

        return regret, eval

In [ ]:
class PosteriorSamplingQLearning(object):
    def __init__(self, H=100, K=20000, c=2, zero_var_start=False, env_type='FrozenLake',env_params={}):
        self.Env = Env_mapping[env_type](H=H,env_params=env_params)
        self.env = self.Env.base_env
        self.A = self.Env.A
        self.S = self.Env.S
        self.H = self.Env.H
        self.K = K
        self.max_V = self.H if env_params.get('penalty_case',False) else 1

        # predefine the maximum V you can get here
        self.T = self.K * self.H
        self.Q = np.ones([self.H, self.S, self.A]) * self.max_V #initialize with that maximum
        # self.Q[self.H] = np.zeros([self.S,self.A])
        self.V = np.ones([self.H + 1, self.S]) * self.max_V #initialize with that maximum
        self.V[self.H] = np.zeros(self.S)
        if env_params.get('discounted_reward', False):
            for s in self.Env.absorbing_states:
                self.Q[:, int(s), :] = np.zeros([self.H, self.A])
                self.V[:, int(s)] = np.zeros(self.H + 1)
        self.V_bar = self.V.copy()
        self.N = np.zeros([self.H, self.S, self.A])
        self.N_last = np.zeros(self.S)
        ## todo the parameter setting for b and c

        self.c = c # the multiplicative a for variance
        self.b = 1
        self.p = .05 # the \delta in script
        self.gamma = 1 if not env_params.get('discounted_reward',False) else env_params.get('gamma',0.99)
        self.J = self.b * np.log( self.S * self.A * self.H * self.T / self.p)
        self.logging_Q = {}
        self.zero_var_start = zero_var_start
        self.no_early_stopping = (env_params.get('circle_mdp', False) or (not env_params.get('discounted_reward', False)) )
        self.env_params = env_params
        self.env_type = env_type

    def learnRate(self, t):
        return (self.H + 1) / (self.H + t)



    def train(self):
        self.rList = []
        self.rList_stepwise = []

        sample_size = int(self.J)
        # for h in range(self.H):
        #     for s in range(self.S):
        #       for a in range(self.A):
        #         self.Q[h,s,a] += self.Bonusterm(1, h, s, a, s)
        for k in tqdm(range(self.K)):
            s = self.env.reset()
            d = False
            rEpisode = 0
            trajectory = {}
            sample_trajectory = {}
            for h in range(self.H):
                # compute your variance with H*(V_max)^2 instead of H^3
                var = self.c*( (self.max_V ** 2)) / np.maximum(np.ones(self.A), self.N[h,s,:])
                Q_sample = self.Q[h,s,:] + np.random.randn(self.A) * np.sqrt(var)
                a = np.argmax(Q_sample)
                s1, r, d, _ = self.env.step(a)
                if self.env_type == 'FrozenLake' and d and self.env_params.get('circle_mdp', False) and s == self.S-1:
                  s1 = self.env.reset()
                if self.env_type == 'FrozenLake' and d and not self.env_params.get('discounted_reward', False) and s == s1 == self.S-1:
                  r = self.rList_stepwise[-1]
                if d and self.env_params.get('discounted_reward', False) and s1 == self.S-1:
                  r = (self.H - h) / self.H
                rEpisode += r*(self.gamma**h)
                self.rList_stepwise.append(r*(self.gamma**h))
                trajectory[h] = (s,a,s1,r,d)
                sample_trajectory[h,s] = Q_sample
                self.N[h, s, a] += 1
                s = s1


                if d and not self.no_early_stopping:
                    break

            if k%10000 == 0:
                print([trajectory[h][0] for h in range(self.H) if h in trajectory])
            self.N_last[s] += 1
            logging_Q = self.Q.copy()
            for h in range(self.H-1, -1, -1):
                if h in trajectory:
                    s,a,s1,r,d = trajectory[h]
                    t = self.N[h, s, a]
                    lr = self.learnRate(t)
                    # compute your variance with H*(V_max)^2 instead of H^3
                    if h < self.H - 1 and (self.no_early_stopping or s1 not in self.Env.absorbing_states):
                        var1 = self.c*(  (self.max_V ** 2)) / np.maximum(np.ones(self.A), self.N[h+1,s1,:])
                        q_sample = self.Q[h+1,s1,:].reshape(self.A, 1) + np.random.randn(self.A, sample_size) * np.sqrt(var1.reshape(self.A, 1))
                        V_next = min(self.max_V, np.max(q_sample))
                    else:
                        # var1 = self.c*(  (self.max_V ** 2)) / np.maximum(np.ones(self.A), self.N_last[s1])
                        V_next = 0

                    # update your Q with r + V_next only if 1. not terminated 2. current h<=H, else update it with r only
                    self.Q[h, s, a] = (1 - lr) * self.Q[h, s, a] + lr * (r + self.gamma * V_next)
                    logging_Q[h] = self.Q[h]
                    logging_Q[h][s] = sample_trajectory[h,s]

                    self.V[h, s] = min(np.max(self.Q[h, s, :]), self.max_V)


            # self.logging_Q[k] = logging_Q
            self.rList.append(rEpisode)
        self.trajectory = trajectory

    def stepwise_regret(self):
        self.regret_step_wise = []
        self.Env.compute_transition()
        self.Env.value_iteration()
        self.pi_star = self.Env.Q_prime.argmax(axis=2)
        total_steps = len(self.rList_stepwise)
        current_steps = 0
        current_cum_regret = 0
        while current_steps < total_steps:
            s = self.env.reset()
            # d = False
            for h in range(self.H):
                a = int(self.pi_star[h][s])
                s1, r, d, _ = self.env.step(a)
                if d and self.env_params.get('discounted_reward', False) and s1 == self.S-1:
                    r = (self.H - h) / self.H
                current_cum_regret += r*(self.gamma**h) - self.rList_stepwise[current_steps]
                if current_steps % self.H == 0:
                    self.regret_step_wise.append(current_cum_regret)
                s = s1
                current_steps += 1
                if d and not self.no_early_stopping:
                    break
                if current_steps >= total_steps:
                    break

    def regret_compute(self):
        regret = []
        self.Env.compute_transition()
        self.Env.value_iteration()
        V_prime = self.Env.V_prime
        for k in tqdm(range(self.K)):
            Q_k = self.logging_Q[k]
            policy_k = Q_k.argmax(axis=2)
            eval = self.Env.policy_evaluation(policy_k)
            if regret:
                regret.append(regret[k-1]+ (V_prime[0][0] - eval[0][0]))
            else:
                regret.append( (V_prime[0][0] - eval[0][0]))

        return regret

In [ ]:
class RLSVI(object):
    def __init__(self, H=100, K=20000, c=2, zero_var_start=False, env_type='FrozenLake',env_params={}):
        self.Env = Env_mapping[env_type](H=H,env_params=env_params)
        self.env = self.Env.base_env
        self.A = self.Env.A
        self.S = self.Env.S
        self.H = self.Env.H
        self.K = K
        self.max_V = self.H if env_params.get('penalty_case',False) else 1

        # predefine the maximum V you can get here
        self.T = self.K * self.H
        self.beta = {k: .5 * c * (self.max_V**2) *np.log(2 * self.H * self.S * self.A * (k+1)) for k in range(self.K)} #* self.H
        self.Q = np.random.randn(self.H + 1, self.S, self.A) * np.sqrt(self.beta[0]) #initialize with that maximum
        self.Q[self.H] = np.zeros([self.S,self.A])
        self.V = np.clip(self.Q.max(axis=2),0,self.max_V)
        if env_params.get('discounted_reward', False):
            for s in self.Env.absorbing_states:
                self.Q[:, int(s),:] = np.zeros([self.H + 1, self.A])
                self.V[:, int(s)] = np.zeros(self.H+1)
        self.V_bar = self.V.copy()
        self.N = np.zeros([self.H, self.S, self.A])
        self.D = np.zeros([self.H+1, self.S, self.A ,self.S+1])
        self.c = c
        self.gamma = 1 if not env_params.get('discounted_reward',False) else env_params.get('gamma',0.99)
        self.logging_Q = {}
        self.zero_var_start = zero_var_start
        self.no_early_stopping = (env_params.get('circle_mdp', False) or (not env_params.get('discounted_reward', False)) )
        self.env_params = env_params
        self.env_type = env_type

    def train(self):
        self.rList = []
        self.rList_stepwise = []
        self.logging_Q[0] = self.Q.copy()
        for k in tqdm(range(1, self.K)):
            s = self.env.reset()
            d = False
            rEpisode = 0
            trajectory = {}
            r_k = 0
            for h in range(self.H):
                a = np.random.choice(np.where(self.Q[h,s,:] == self.Q[h,s,:].max())[0])
                self.N[h,s,a] += 1
                s1, r, d, _ = self.env.step(a)
                if self.env_type == 'FrozenLake' and d and self.env_params.get('circle_mdp', False) and s == self.S-1:
                  s1 = self.env.reset()
                if self.env_type == 'FrozenLake'  and d and not self.env_params.get('discounted_reward', False) and  s == s1 == self.S-1:
                  r = self.rList_stepwise[-1]
                if d and self.env_params.get('discounted_reward', False) and s1 == self.S-1:
                  r = (self.H - h) / self.H

                self.D[h][s][a][0] += r
                self.D[h][s][a][int(s1) + 1] += 1
                trajectory[h] = (s,a,s1,r,d)
                r_k += r*(self.gamma**h)
                self.rList_stepwise.append(r*(self.gamma**h))
                if d and not self.no_early_stopping:
                  break
                s = s1
            if k%10000 == 0:
                print([trajectory[h][0] for h in range(self.H) if h in trajectory])
            # if self.env_params.get('discounted_reward', False):
            #     for h in range(self.H-1, -1, -1):
            #         if h in trajectory:
            #             s,a,s1,r,d = trajectory[h]
            #             PV_next = (self.D[h][s][a][1:] @ self.V[h+1]) / (self.N[h][s][a] + 1)
            #             self.Q[h][s][a] = PV_next + self.D[h][s][a][0]  / (self.N[h][s][a] + 1) + np.random.randn() * np.sqrt(self.beta[k] / (self.N[h][s][a] + 1))
            #             self.V[h][s] = np.max(self.Q[h][s])
            # else:
            for h in range(self.H-1, -1, -1):
                PV_next = (self.D[h,:,:,1:] @ self.V[h+1] ) / (self.N[h] + 1)
                self.Q[h] = PV_next + self.D[h,:,:,0] /  (self.N[h] + 1) + np.random.randn(self.S, self.A) * np.sqrt(self.beta[k] / (self.N[h] + 1))
                if self.env_params.get('discounted_reward', False):
                    for s in self.Env.absorbing_states:
                        self.Q[h, int(s),:] = np.zeros(self.A)
                self.V[h] = np.clip(np.max(self.Q[h],axis=1), 0 , (self.H - h)/self.H)

            logging_Q = self.Q.copy()
            # self.logging_Q[k] = logging_Q
            self.rList.append(r_k)
        self.trajectory = trajectory

    def stepwise_regret(self):
        self.regret_step_wise = []
        self.Env.compute_transition()
        self.Env.value_iteration()
        self.pi_star = self.Env.Q_prime.argmax(axis=2)
        total_steps = len(self.rList_stepwise)
        current_steps = 0
        current_cum_regret = 0
        while current_steps < total_steps:
            s = self.env.reset()
            # d = False
            for h in range(self.H):
                a = int(self.pi_star[h][s])
                s1, r, d, _ = self.env.step(a)
                if d and self.env_params.get('discounted_reward', False) and s1 == self.S-1:
                    r = (self.H - h) / self.H
                current_cum_regret += r*(self.gamma**h) - self.rList_stepwise[current_steps]
                if current_steps % self.H == 0:
                    self.regret_step_wise.append(current_cum_regret)
                s = s1
                current_steps += 1
                if d and not self.no_early_stopping:
                    break
                if current_steps >= total_steps:
                    break

    def regret_compute(self):
        regret = []
        self.Env.compute_transition()
        self.Env.value_iteration()
        V_prime = self.Env.V_prime
        for k in tqdm(range(self.K)):
            Q_k = self.logging_Q[k]
            policy_k = Q_k.argmax(axis=2)
            eval = self.Env.policy_evaluation(policy_k)
            if regret:
                regret.append(regret[k-1]+ (V_prime[0][0] - eval[0][0]))
            else:
                regret.append( (V_prime[0][0] - eval[0][0]))

        return regret

In [ ]:
class LSVI_UCB(object):
    def __init__(self, H=25, K=20000, c=0.01, env_type = 'FrozenLake', env_params={}):
        self.Env = Env_mapping[env_type](H=H, env_params=env_params)
        self.env_type = env_type
        self.env = self.Env.base_env
        self.A = self.Env.A
        self.S = self.Env.S
        self.H = self.Env.H
        self.d = self.S * self.A
        self.K = K
        self.T = self.K * self.H
        self.max_V = self.H if env_params.get('penalty_case',False) else 1
        self.c = c # need more tuning
        self.p = .05 # need more tuning
        self.tau = np.log(2 * self.d  *self.T / self.p)
        self.gamma = 1 if not env_params.get('discounted_reward',False) else env_params.get('gamma',0.99)
        self.env_params = env_params
        self.no_early_stopping = (env_params.get('circle_mdp', False) or (not env_params.get('discounted_reward', False)) )
        self.lamb = 1
        self.beta = self.c * self.max_V * np.sqrt(self.tau) # * self.d  do we need this d?

    def train(self):
        self.N = np.ones([self.H, self.S, self.A]) * self.lamb
        self.D = np.zeros([self.H, self.S, self.A, self.S+1])
        self.Q = np.ones([self.H, self.S, self.A]) * self.max_V
        self.V = np.ones([self.H + 1, self.S]) * self.max_V
        self.V[self.H] = np.zeros(self.S) # This one is the must
        if self.env_params.get('discounted_reward', False):
            for s in self.Env.absorbing_states:
                self.V[:, int(s)] = np.zeros(self.H + 1)

        self.rList = []
        # self.rList_stepwise = []
        for k in tqdm(range(self.K)):
            s = self.env.reset()
            d = False
            r_k = 0
            trajectory = {}
            for h in range(self.H):
                a = np.random.choice(np.where(self.Q[h][s] == self.Q[h][s].max())[0])
                s1, r, d, _ = self.env.step(a)
                if self.env_type == 'FrozenLake' and d and self.env_params.get('circle_mdp', False) and s == self.S-1:
                  s1 = self.env.reset()
                if self.env_type == 'FrozenLake'  and d and not self.env_params.get('discounted_reward', False) and  s == s1 == self.S-1:
                  r = self.rList_stepwise[-1]
                if d and self.env_params.get('discounted_reward', False) and s1 == self.S-1:
                  r = (self.H - h) / self.H
                trajectory[h] = (s,a,s1,r,d)
                self.N[h,s,a] += 1
                self.D[h][s][a][0] += r
                self.D[h][s][a][s1 + 1] += 1

                r_k += r*(self.gamma**h)
                # self.rList_stepwise.append(r*(self.gamma**h))
                if d and not self.no_early_stopping:
                  break
                s = s1


            if k%10000 == 0:
                print([trajectory[h][0] for h in range(self.H) if h in trajectory])
            for h in range(self.H-1, -1, -1):
                r_PV_next = (self.D[h,:,:,1:] @ self.V[h+1] ) + self.D[h,:,:,0]
                self.Q[h] = np.minimum(r_PV_next/ self.N[h] + self.beta * np.sqrt(1/self.N[h]), self.max_V)
                if self.env_params.get('discounted_reward', False):
                    for s in self.Env.absorbing_states:
                        self.Q[h, int(s),:] = np.zeros(self.A)
                self.V[h] = np.clip(np.max(self.Q[h],axis=1), 0 , (self.H - h)/self.H)

            self.rList.append(r_k)

In [ ]:
def test_on_params(H=15,K=20000,c=0.01,learning_type='Hoeffding',env_type='FrozenLake',env_params={},eval_V_pi = False,experiment_name_suffix=""):
    env_p = env_params.get('drift_probability',0.85) if env_type == 'DrunkMan' else 0.33
    absorbing_state = 'No' if env_params.get('simple_example', True) else 'Many'
    penalty_case = env_params.get('penalty_case',False)
    gamma = 1 if not env_params.get('discounted_reward',False) else env_params.get('gamma',0.99)
    respawn = env_params.get('circle_mdp',False)
    state_space = env_params.get('space_length', 7) if env_type=='DrunkMan' else  len(env_params.get('desc_map', 4))**2
    wandb.init(
    # set the wandb project where this run will be logged
    project="PEQL_1128_F",
    name = f'{learning_type},env:{env_type},env_p={env_p},penalty_case:{penalty_case},state_space_size:{state_space}',
    # track hyperparameters and run metadata
    config={
    "H": H,
    "Algo_name" : learning_type,
    "c":c,
    "b":1,
    "env_p": env_params.get('drift_probability',0.85) if env_type == 'DrunkMan' else 0.33,
    'p':0.05,
    'K':K,
    "lr_type": "adaptive",
    "Env_name": env_type,
    'penalty_case':env_params.get('penalty_case',False),
    'env_no_absorbing_state':env_params.get('simple_example', True),
    'discounted_reward':env_params.get('discounted_reward',False),
    'gamma':1 if not env_params.get('discounted_reward',False) else env_params.get('gamma',0.99)
    }
)
    if learning_type == 'Hoeffding':
        Q_learning = ProvablyEfficientQLearning(H=H, K=K, c=np.sqrt(c),env_type=env_type,env_params=env_params)
    elif learning_type == 'Posterior Sampling':
        Q_learning = PosteriorSamplingQLearning(H=H,K=K,c=c,env_type=env_type,env_params=env_params)
    elif learning_type == 'RLSVI':
        Q_learning = RLSVI(H=H,K=K,c=c,env_type=env_type,env_params=env_params)
    elif learning_type == 'LSVI_UCB':
        Q_learning = LSVI_UCB(H=H,K=K,c=np.sqrt(c),env_type=env_type,env_params=env_params)
    Q_learning.train()
    Q_learning.Env.compute_transition()
    Q_learning.Env.value_iteration()
    V_prime = Q_learning.Env.V_prime
    rList = Q_learning.rList
    cum_regret = 0
    regret_list = []
    cum_reward = 0

    if eval_V_pi:
        regret = Q_learning.regret_compute()

    for idx in range(len(rList)):
        i = rList[idx]
        cum_regret += V_prime[0][0] - i
        regret_list.append(cum_regret)
        cum_reward += i
        if not eval_V_pi:
          wandb.log({'cumulative_regret':cum_regret,'current_episode_regret':V_prime[0][0] - i,'reward':cum_reward })#, 'Q_left':Q_learning.logging_Q[idx][0,0,0], 'Q_right':Q_learning.logging_Q[idx][0,0,1],
                   #'Q_left_last':Q_learning.logging_Q[idx][-2,-2,0], 'Q_right_last':Q_learning.logging_Q[idx][-2,-2,1]})
        else:
          wandb.log({'cumulative_regret':regret[idx],'cumulative_regret_with_reward':cum_regret,'current_episode_regret':V_prime[0][0] - i,'reward':cum_reward, 'Q_left':Q_learning.logging_Q[idx][0,0,0], 'Q_right':Q_learning.logging_Q[idx][0,0,1],
                   'Q_left_last':Q_learning.logging_Q[idx][-2,-2,0], 'Q_right_last':Q_learning.logging_Q[idx][-2,-2,1]})
    # if not Q_learning.no_early_stopping:
    #     Q_learning.stepwise_regret()
    #     stepwise_regret_list = Q_learning.regret_step_wise
    #     for idx in range(len(stepwise_regret_list)):
    #         wandb.log({'cumulative_stepwise_regret':stepwise_regret_list[idx]})
    return regret_list